In [81]:
import torch
from transformers import BartTokenizer, BartForConditionalGeneration
from datasets import load_dataset
from sklearn.metrics import accuracy_score
import sentencepiece
from langchain_ollama import ChatOllama
from langchain.schema import HumanMessage

In [75]:
model = ChatOllama(model="llama3.1", temperature=0.0,device='mps')

In [5]:
dataset = load_dataset("glue", "mrpc")

In [69]:
def add_hard_prompts(sentence1, sentence2):
    """
    This function adds a hard prompt to each input example
    to make the task explicit to the model.
    """
    prompt = f"""Paraphrase detection: Does the following pair of sentences mean the same thing? Sentence 1: "{sentence1}" Sentence 2: "{sentence2}". Reply with a single word answer: 'yes' if they mean the same thing, otherwise 'no'."""
    return prompt

In [71]:
validation_prompts = [add_hard_prompts(sent1, sent2) for sent1, sent2 in zip(dataset["validation"]["sentence1"], dataset["validation"]["sentence2"])]

In [72]:
out=[]

In [92]:
def llama_inference(prompts, model):
    """
    This function uses LLaMA 3.1 model for inference on hard-prompted inputs.
    """
    predictions = []
    for prompt in prompts:
        # Convert prompt to HumanMessage
        human_message = HumanMessage(content=prompt)
        
        # Query the LLaMA 3.1 model using __call__
        response = model([human_message])

        
        # Extract the response text (response is already an AIMessage object)
        prediction = response.content.strip().lower()
        out.append(prediction)
        # Convert textual output to binary labels
        if "yes" in prediction:
            predictions.append(1)  # Paraphrase
        else:
            predictions.append(0)  # Not a paraphrase
            
    return predictions


In [93]:
predictions = llama_inference(validation_prompts, model)

In [126]:
print("Example Prompt and Prediction:")
print(validation_prompts[4])
print(predictions[4])
print(out[4])

Example Prompt and Prediction:
Paraphrase detection: Does the following pair of sentences mean the same thing? Sentence 1: "No dates have been set for the civil or the criminal trial ." Sentence 2: "No dates have been set for the criminal or civil cases , but Shanley has pleaded not guilty .". Reply with a single word answer: 'yes' if they mean the same thing, otherwise 'no'.
1
yes.


In [127]:
from sklearn.metrics import classification_report, accuracy_score

# Get true labels from the validation set
true_labels = dataset["validation"]["label"]

# Print classification report
print("Classification Report:")
print(classification_report(true_labels, predictions))

# Print overall accuracy
accuracy = accuracy_score(true_labels, predictions)
print(f"Validation Accuracy with Hard Prompts: {accuracy:.4f}")


Classification Report:
              precision    recall  f1-score   support

           0       0.61      0.43      0.51       129
           1       0.77      0.87      0.82       279

    accuracy                           0.73       408
   macro avg       0.69      0.65      0.66       408
weighted avg       0.72      0.73      0.72       408

Validation Accuracy with Hard Prompts: 0.7328
